In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import time as time_module
import json
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

class KalshiWeatherAnalyzer:
    """
    Analyzes Kalshi weather market efficiency for 2024
    Creates two CSV files:
    1. 9PM Analysis: Most popular temperature choice for each city at 9PM the day before
    2. 7AM Analysis: Most popular city at 7AM the day of the event
    """
    
    def __init__(self):
        self.base_url = "https://api.elections.kalshi.com/trade-api/v2"
        self.cities = {
            'NY': 'KXHIGHNY',
            'LA': 'KXHIGHLA', 
            'PHIL': 'KXHIGHPHIL',
            'CHI': 'KXHIGHCHI',
            'MIA': 'KXHIGHMIA',
            'DEN': 'KXHRIGHDEN'
        }
        
        # Results storage
        self.pm9_results = []  # 9PM day before analysis
        self.am7_results = []  # 7AM day of analysis
        
    def create_event_ticker(self, city: str, date: datetime) -> str:
        """Create event ticker from city and date"""
        city_prefix = f"HIGH{city}"
        date_str = date.strftime("%y%b%d").upper()
        return f"{city_prefix}-{date_str}"
    
    def datetime_to_timestamp(self, dt: datetime) -> int:
        """Convert datetime to Unix timestamp"""
        return int(dt.timestamp())
    
    def get_event_markets(self, event_ticker: str) -> Optional[Dict]:
        """Get all temperature bracket markets for an event"""
        try:
            url = f"{self.base_url}/events/{event_ticker}"
            response = requests.get(url, headers={'accept': 'application/json'})
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Failed to get event {event_ticker}: {response.status_code}")
                return None
        except Exception as e:
            print(f"Error getting event {event_ticker}: {str(e)}")
            return None
    
    def get_market_candlesticks(self, series_ticker: str, market_ticker: str, 
                              start_ts: int, end_ts: int) -> Optional[Dict]:
        """Get candlestick data for a specific market"""
        try:
            url = f"{self.base_url}/series/{series_ticker}/markets/{market_ticker}/candlesticks"
            params = {
                'start_ts': start_ts,
                'end_ts': end_ts,
                'period_interval': 60  # 1-minute intervals
            }
            
            response = requests.get(url, params=params, headers={'accept': 'application/json'})
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Failed to get candlesticks for {market_ticker}: {response.status_code}")
                return None
        except Exception as e:
            print(f"Error getting candlesticks for {market_ticker}: {str(e)}")
            return None
    
    def find_exact_candlestick(self, target_ts: int, candlesticks: List[Dict]) -> Optional[Dict]:
        """Find candlestick with exact timestamp match, or closest if no exact match"""
        if not candlesticks:
            return None
        
        # First try to find exact match
        for candle in candlesticks:
            if candle.get('end_period_ts') == target_ts:
                print(f"    Found exact timestamp match: {target_ts}")
                return candle
        
        # If no exact match, find closest
        closest = min(candlesticks, key=lambda x: abs(x['end_period_ts'] - target_ts))
        print(f"    No exact match for {target_ts}, using closest: {closest.get('end_period_ts')}")
        return closest
    
    def get_final_ask_price(self, series_ticker: str, market_ticker: str) -> Optional[float]:
        """Get the final ask price from the very last candlestick (yes_ask.open)"""
        try:
            # Parse the event date from ticker to get appropriate time range
            event_parts = market_ticker.split('-')
            if len(event_parts) >= 2:
                date_part = event_parts[1]  # e.g., "24JUL20"
                year = int('20' + date_part[:2])
                month_str = date_part[2:5]
                day = int(date_part[5:])
                
                month_map = {
                    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
                    'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
                }
                
                if month_str in month_map:
                    event_date = datetime(year, month_map[month_str], day)
                    
                    # Get data for the event day and several days after for resolution
                    start_time = event_date
                    end_time = event_date + timedelta(days=10)  # Markets can take time to resolve
                    
                    print(f"    Getting complete series for {market_ticker}")
                    print(f"    Date range: {start_time.strftime('%Y-%m-%d')} to {end_time.strftime('%Y-%m-%d')}")
                    
                    candlestick_data = self.get_market_candlesticks(
                        series_ticker, 
                        market_ticker,
                        self.datetime_to_timestamp(start_time),
                        self.datetime_to_timestamp(end_time)
                    )
                    
                    if candlestick_data and candlestick_data.get('candlesticks'):
                        candlesticks = candlestick_data['candlesticks']
                        print(f"    Found {len(candlesticks)} candlesticks")
                        
                        if candlesticks:
                            # Get the LAST candlestick (most recent)
                            last_candle = candlesticks[-1]
                            last_timestamp = last_candle.get('end_period_ts')
                            
                            # Convert timestamp to readable date
                            last_time = datetime.fromtimestamp(last_timestamp) if last_timestamp else "Unknown"
                            
                            if last_candle and last_candle.get('yes_ask'):
                                final_ask = last_candle['yes_ask'].get('open')
                                print(f"    Last candlestick: timestamp {last_timestamp} ({last_time})")
                                print(f"    Final yes_ask.open: ${final_ask}")
                                return final_ask
                            else:
                                print(f"    No yes_ask data in last candlestick")
                    else:
                        print(f"    No candlestick data found for final price")
            
            return None
        except Exception as e:
            print(f"    Error getting final ask for {market_ticker}: {str(e)}")
            return None
    
    def get_final_ask_price_for_city(self, city: str, date: datetime) -> Optional[float]:
        """Get the final ask price for the most popular market of a city on a given date"""
        try:
            event_ticker = self.create_event_ticker(city, date)
            series_ticker = self.cities[city]
            event_data = self.get_event_markets(event_ticker)
            
            if not event_data or not event_data.get('markets'):
                return None
            
            # Find the market with lowest ask (most popular)
            best_final_ask = None
            min_ask = float('inf')
            
            for market in event_data['markets']:
                market_ticker = market['ticker']
                final_ask = self.get_final_ask_price(series_ticker, market_ticker)
                
                if final_ask and final_ask < min_ask:
                    min_ask = final_ask
                    best_final_ask = final_ask
            
            return best_final_ask
        except Exception as e:
            print(f"Error getting final ask for city {city}: {str(e)}")
            return None

    def analyze_9pm_popular_choice(self, city: str, date: datetime) -> Optional[Dict]:
        """
        Analyze most popular temperature choice at 9PM the day before
        Returns: {city, date, popular_choice, last_ask, outcome}
        """
        # 9PM the day before (UTC time)
        target_time = datetime.combine(date - timedelta(days=1), time(21, 0))
        target_ts = self.datetime_to_timestamp(target_time)
        
        print(f"  9PM target: {target_time} UTC (timestamp: {target_ts})")
        
        event_ticker = self.create_event_ticker(city, date)
        series_ticker = self.cities[city]
        
        # Get all temperature brackets for this event
        event_data = self.get_event_markets(event_ticker)
        if not event_data:
            return None
        
        markets = event_data.get('markets', [])
        if not markets:
            return None
        
        best_market = None
        best_ask = float('inf')
        
        # Find market with lowest ask price (most popular/likely)
        for market in markets:
            market_ticker = market['ticker']
            
            # Get candlestick data around 9PM with wider range
            start_ts = target_ts - 14400  # 4 hours before
            end_ts = target_ts + 14400    # 4 hours after
            
            candlestick_data = self.get_market_candlesticks(
                series_ticker, market_ticker, start_ts, end_ts
            )
            
            if candlestick_data and candlestick_data.get('candlesticks'):
                # Find exact or closest candlestick to 9PM
                closest = self.find_exact_candlestick(target_ts, candlestick_data['candlesticks'])
                
                if closest and closest.get('yes_ask') and closest['yes_ask'].get('open') is not None:
                    current_ask = closest['yes_ask']['open']  # Use OPEN price
                    timestamp = closest.get('end_period_ts')
                    print(f"    Market {market_ticker}: timestamp {timestamp}, yes_ask.open: ${current_ask}")
                    
                    if current_ask < best_ask and current_ask > 0:  # Ensure valid price
                        best_ask = current_ask
                        best_market = {
                            'market': market,
                            'ask_price': current_ask,
                            'temperature_range': market.get('subtitle', 'Unknown'),
                            'ticker': market_ticker,
                            'timestamp': timestamp
                        }
        
        if best_market:
            # Get the final ask price from the market data (latest available)
            print(f"    Getting final price for best market: {best_market['ticker']}")
            final_ask = self.get_final_ask_price(series_ticker, best_market['ticker'])
            
            # Determine outcome based on final ask price
            outcome_9pm = 'TRUE' if best_market['ask_price'] > 95 else ('FALSE' if best_market['ask_price'] < 5 else 'UNCERTAIN')
            outcome_final = 'TRUE' if final_ask and final_ask > 95 else ('FALSE' if final_ask and final_ask < 5 else 'UNCERTAIN')
            
            return {
                'city': city,
                'date': date.strftime('%Y-%m-%d'),
                'popular_choice': best_market['temperature_range'],
                'ask_at_9pm': best_market['ask_price'],
                'final_ask': final_ask,
                'outcome_9pm': outcome_9pm,
                'outcome_final': outcome_final,
                'ticker': best_market['ticker']
            }
        
        return None
    
    def analyze_7am_popular_city(self, date: datetime) -> Optional[Dict]:
        """
        Analyze most popular city at 7AM the day of
        Returns: {date, popular_city, last_ask, outcome}
        """
        # 7AM the day of (UTC time)
        target_time = datetime.combine(date, time(7, 0))
        target_ts = self.datetime_to_timestamp(target_time)
        
        print(f"  7AM target: {target_time} UTC (timestamp: {target_ts})")
        
        city_popularity = {}
        
        # Check each city
        for city, series_ticker in self.cities.items():
            event_ticker = self.create_event_ticker(city, date)
            event_data = self.get_event_markets(event_ticker)
            
            if not event_data or not event_data.get('markets'):
                print(f"    No event data for {city}")
                continue
            
            print(f"    Checking {city} with {len(event_data['markets'])} markets")
            
            # Find the market with highest volume or lowest ask (most popular)
            total_volume = 0
            min_ask = float('inf')
            markets_with_data = 0
            
            for market in event_data['markets']:
                market_ticker = market['ticker']
                
                # Get candlestick data around 7AM with wider range
                start_ts = target_ts - 14400  # 4 hours before
                end_ts = target_ts + 14400    # 4 hours after
                
                candlestick_data = self.get_market_candlesticks(
                    series_ticker, market_ticker, start_ts, end_ts
                )
                
                if candlestick_data and candlestick_data.get('candlesticks'):
                    closest = self.find_exact_candlestick(target_ts, candlestick_data['candlesticks'])
                    
                    if closest:
                        markets_with_data += 1
                        volume = closest.get('volume', 0)
                        total_volume += volume
                        timestamp = closest.get('end_period_ts')
                        
                        if closest.get('yes_ask') and closest['yes_ask'].get('open') is not None:
                            current_ask = closest['yes_ask']['open']  # Use OPEN price
                            print(f"      Market {market_ticker}: timestamp {timestamp}, yes_ask.open: ${current_ask}, volume: {volume}")
                            if current_ask > 0:  # Ensure valid price
                                min_ask = min(min_ask, current_ask)
            
            print(f"    {city}: {markets_with_data} markets with data, total volume: {total_volume}, min ask: {min_ask}")
            
            if min_ask < float('inf') and markets_with_data > 0:
                city_popularity[city] = {
                    'volume': total_volume,
                    'min_ask': min_ask,
                    'score': total_volume / (min_ask + 1),  # Higher volume, lower ask = more popular
                    'markets_count': markets_with_data
                }
        
        print(f"  City popularity scores: {city_popularity}")
        
        if city_popularity:
            # Find most popular city
            popular_city = max(city_popularity.items(), key=lambda x: x[1]['score'])
            city_name = popular_city[0]
            city_data = popular_city[1]
            
            print(f"  Most popular city: {city_name} (score: {city_data['score']:.2f})")
            
            # Get final ask price for the most popular city's market
            final_ask = self.get_final_ask_price_for_city(city_name, date)
            
            # Determine outcomes
            outcome_7am = 'TRUE' if city_data['min_ask'] > 95 else ('FALSE' if city_data['min_ask'] < 5 else 'UNCERTAIN')
            outcome_final = 'TRUE' if final_ask and final_ask > 95 else ('FALSE' if final_ask and final_ask < 5 else 'UNCERTAIN')
            
            return {
                'date': date.strftime('%Y-%m-%d'),
                'popular_city': city_name,
                'ask_at_7am': city_data['min_ask'],
                'final_ask': final_ask,
                'outcome_7am': outcome_7am,
                'outcome_final': outcome_final
            }
        
        print(f"  No city popularity data found")
        return None
    
    def process_date_range(self, start_date: datetime, end_date: datetime):
        """Process all dates in the given range"""
        current_date = start_date
        total_days = (end_date - start_date).days + 1
        processed_days = 0
        
        print(f"Processing {total_days} days from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
        
        while current_date <= end_date:
            processed_days += 1
            print(f"Processing {current_date.strftime('%Y-%m-%d')} ({processed_days}/{total_days})")
            
            # 9PM Analysis - Most popular choice for each city
            for city in self.cities.keys():
                print(f"  Analyzing 9PM for {city}...")
                result_9pm = self.analyze_9pm_popular_choice(city, current_date)
                if result_9pm:
                    self.pm9_results.append(result_9pm)
                    print(f"    Found: {result_9pm['popular_choice']}")
                    print(f"    9PM Ask: ${result_9pm['ask_at_9pm']}, Final Ask: ${result_9pm['final_ask']}")
                    print(f"    9PM Outcome: {result_9pm['outcome_9pm']}, Final Outcome: {result_9pm['outcome_final']}")
                else:
                    print(f"    No data found for {city}")
            
            # 7AM Analysis - Most popular city
            print(f"  Analyzing 7AM popular city...")
            result_7am = self.analyze_7am_popular_city(current_date)
            if result_7am:
                self.am7_results.append(result_7am)
                print(f"    Popular city: {result_7am['popular_city']}")
                print(f"    7AM Ask: ${result_7am['ask_at_7am']}, Final Ask: ${result_7am['final_ask']}")
                print(f"    7AM Outcome: {result_7am['outcome_7am']}, Final Outcome: {result_7am['outcome_final']}")
            else:
                print(f"    No 7AM data found")
            
            # Rate limiting - small delay between requests
            time_module.sleep(0.5)  # Increased delay to be more API-friendly
            
            current_date += timedelta(days=1)
    
    def save_results(self):
        """Save results to CSV files"""
        # 9PM Results CSV
        if self.pm9_results:
            df_9pm = pd.DataFrame(self.pm9_results)
            df_9pm.to_csv('kalshi_weather_9pm_analysis_2024.csv', index=False)
            print(f"Saved 9PM analysis with {len(self.pm9_results)} records to 'kalshi_weather_9pm_analysis_2024.csv'")
        
        # 7AM Results CSV  
        if self.am7_results:
            df_7am = pd.DataFrame(self.am7_results)
            df_7am.to_csv('kalshi_weather_7am_analysis_2024.csv', index=False)
            print(f"Saved 7AM analysis with {len(self.am7_results)} records to 'kalshi_weather_7am_analysis_2024.csv'")
    
    def discover_available_markets(self) -> Dict:
        """
        Use the markets endpoint to discover what weather markets actually exist
        This is more efficient than trying individual event endpoints
        """
        try:
            # Try the markets endpoint to see what's available
            url = f"{self.base_url}/markets"
            params = {
                'limit': 1000,  # Get a large batch
                'status': 'open',  # Only active markets
                'category': 'Climate and Weather'  # Filter to weather markets
            }
            
            response = requests.get(url, params=params, headers={'accept': 'application/json'})
            
            if response.status_code == 200:
                data = response.json()
                markets = data.get('markets', [])
                
                # Filter for HIGH temperature markets
                weather_markets = []
                for market in markets:
                    ticker = market.get('ticker', '')
                    if 'HIGH' in ticker and any(city in ticker for city in ['NY', 'LA', 'PHIL', 'CHI', 'MIA', 'DEN']):
                        weather_markets.append(market)
                
                print(f"Found {len(weather_markets)} weather markets")
                
                # Group by event
                events_by_date = {}
                for market in weather_markets:
                    event_ticker = market.get('event_ticker', '')
                    if event_ticker:
                        if event_ticker not in events_by_date:
                            events_by_date[event_ticker] = []
                        events_by_date[event_ticker].append(market)
                
                return {
                    'total_markets': len(weather_markets),
                    'events': events_by_date,
                    'raw_markets': weather_markets
                }
            else:
                print(f"Failed to get markets: {response.status_code}")
                return {}
                
        except Exception as e:
            print(f"Error discovering markets: {str(e)}")
            return {}
    
    def test_single_event(self, city: str = 'NY', test_date: str = '2024-07-20'):
        """Test function to verify API calls work with a known event"""
        date_obj = datetime.strptime(test_date, '%Y-%m-%d')
        event_ticker = self.create_event_ticker(city, date_obj)
        
        print(f"Testing with event: {event_ticker}")
        
        # Test event data retrieval
        event_data = self.get_event_markets(event_ticker)
        if event_data:
            print(f"✓ Found {len(event_data.get('markets', []))} markets")
            
            # Test first market candlestick data
            if event_data.get('markets'):
                first_market = event_data['markets'][0]
                print(f"Testing market: {first_market['ticker']}")
                
                # Get candlestick data for a day
                start_time = datetime.combine(date_obj - timedelta(days=1), time(0, 0))
                end_time = datetime.combine(date_obj, time(23, 59))
                
                candlestick_data = self.get_market_candlesticks(
                    self.cities[city], 
                    first_market['ticker'],
                    self.datetime_to_timestamp(start_time),
                    self.datetime_to_timestamp(end_time)
                )
                
                if candlestick_data:
                    print(f"✓ Found {len(candlestick_data.get('candlesticks', []))} candlesticks")
                    return True
                else:
                    print("✗ No candlestick data found")
        else:
            print("✗ No event data found")
        
        return False
    
    def run_full_analysis_2024(self):
        """Run the complete analysis for 2024 (available data)"""
        start_date = datetime(2024, 1, 1)
        end_date = datetime(2024, 10, 31)  # Only go to October 2024 based on data availability
        
        print("Starting Kalshi Weather Market Analysis for 2024...")
        print("(2025 markets likely don't exist yet)")
        self.process_date_range(start_date, end_date)
        self.save_results()
        
        print("\nAnalysis Summary:")
        print(f"9PM Analysis: {len(self.pm9_results)} records")
        print(f"7AM Analysis: {len(self.am7_results)} records")

# Initialize and run the analyzer
analyzer = KalshiWeatherAnalyzer()

# Test API connectivity first
print("Testing API connectivity with known 2024 event...")
if analyzer.test_single_event('NY', '2024-07-20'):
    print("✓ API test successful!")
    
    print("\n" + "="*60)
    print("OPTION 1: DISCOVERY-BASED ANALYSIS (RECOMMENDED)")
    print("="*60)
    print("This will find all available weather markets and analyze them")
    # analyzer.discover_available_markets()  # Uncomment to run discovery
    
    print("\n" + "="*60)
    print("OPTION 2: SPECIFIC DATE RANGE TEST")
    print("="*60)
    print("Testing with a small date range first...")
    
    # Test with just July 20-22, 2024 (known to work)
    test_start = datetime(2024, 7, 20)
    test_end = datetime(2024, 7, 22)
    analyzer.process_date_range(test_start, test_end)
    analyzer.save_results()
    
    print("\n" + "="*60)
    print("OPTION 3: FULL 2024 ANALYSIS")
    print("="*60)
    print("Uncomment the line below to run analysis on ALL of 2024")
    print("(Warning: This will take a long time and make many API calls)")
    print("# analyzer.run_full_analysis_2024()")
    
else:
    print("✗ API test failed - check connectivity and endpoints")

# Uncomment one of these to run different analyses:

# Full 2024 analysis (Jan 1 - Oct 31, 2024):
# analyzer.run_full_analysis_2024()

# Specific month analysis (example: July 2024):
# analyzer.process_date_range(datetime(2024, 7, 1), datetime(2024, 7, 31))

# Summer analysis (June-August 2024):
# analyzer.process_date_range(datetime(2024, 6, 1), datetime(2024, 8, 31))

Testing API connectivity with known 2024 event...
Testing with event: HIGHNY-24JUL20
✓ Found 6 markets
Testing market: HIGHNY-24JUL20-T82
✓ Found 12 candlesticks
✓ API test successful!

OPTION 1: DISCOVERY-BASED ANALYSIS (RECOMMENDED)
This will find all available weather markets and analyze them

OPTION 2: SPECIFIC DATE RANGE TEST
Testing with a small date range first...
Processing 3 days from 2024-07-20 to 2024-07-22
Processing 2024-07-20 (1/3)
  Analyzing 9PM for NY...
  9PM target: 2024-07-19 21:00:00 UTC (timestamp: 1721440800)
    Found exact timestamp match: 1721440800
    Market HIGHNY-24JUL20-T82: timestamp 1721440800, yes_ask.open: $12
    Found exact timestamp match: 1721440800
    Market HIGHNY-24JUL20-B82.5: timestamp 1721440800, yes_ask.open: $15
    No exact match for 1721440800, using closest: 1721437200
    Market HIGHNY-24JUL20-B84.5: timestamp 1721437200, yes_ask.open: $39
    Found exact timestamp match: 1721440800
    Market HIGHNY-24JUL20-B86.5: timestamp 172144080

In [8]:
discovery_data = analyzer.discover_available_markets()

Found 36 weather markets


In [3]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import time as time_module
import json
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

class KalshiWeatherAnalyzer:
    """
    Simple Kalshi weather market efficiency analyzer
    Creates CSV with: City, Date, 9PM_Cost, Ending_Cost, Won
    """
    
    def __init__(self):
        self.base_url = "https://api.elections.kalshi.com/trade-api/v2"
        self.cities = {
            'NY': 'KXHIGHNY',
            'LA': 'KXHIGHLA', 
            'PHIL': 'KXHIGHPHIL',
            'CHI': 'KXHIGHCHI',
            'MIA': 'KXHIGHMIA',
            'DEN': 'KXHIGHDEN'
        }
        
        # Results storage
        self.pm9_ask_results = []  # 9PM highest ASK analysis
        self.pm9_bid_results = []  # 9PM highest BID analysis
        self.am8_ask_results = []  # 8AM highest ASK analysis
        self.am8_bid_results = []  # 8AM highest BID analysis
        
    def create_event_ticker(self, city: str, date: datetime) -> str:
        """Create event ticker from city and date"""
        city_prefix = f"HIGH{city}"
        date_str = date.strftime("%y%b%d").upper()
        return f"{city_prefix}-{date_str}"
    
    def datetime_to_timestamp(self, dt: datetime) -> int:
        """Convert datetime to Unix timestamp"""
        return int(dt.timestamp())
    
    def get_event_markets(self, event_ticker: str) -> Optional[Dict]:
        """Get all temperature bracket markets for an event"""
        try:
            url = f"{self.base_url}/events/{event_ticker}"
            response = requests.get(url, headers={'accept': 'application/json'})
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Failed to get event {event_ticker}: {response.status_code}")
                return None
        except Exception as e:
            print(f"Error getting event {event_ticker}: {str(e)}")
            return None
    
    def get_market_candlesticks(self, series_ticker: str, market_ticker: str, 
                              start_ts: int, end_ts: int) -> Optional[Dict]:
        """Get candlestick data for a specific market"""
        try:
            url = f"{self.base_url}/series/{series_ticker}/markets/{market_ticker}/candlesticks"
            params = {
                'start_ts': start_ts,
                'end_ts': end_ts,
                'period_interval': 60  # 1-minute intervals
            }
            
            response = requests.get(url, params=params, headers={'accept': 'application/json'})
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Failed to get candlesticks for {market_ticker}: {response.status_code}")
                return None
        except Exception as e:
            print(f"Error getting candlesticks for {market_ticker}: {str(e)}")
            return None
    
    def find_exact_candlestick(self, target_ts: int, candlesticks: List[Dict]) -> Optional[Dict]:
        """Find candlestick with exact timestamp match, or closest if no exact match"""
        if not candlesticks:
            return None
        
        # First try to find exact match
        for candle in candlesticks:
            if candle.get('end_period_ts') == target_ts:
                return candle
        
        # If no exact match, find closest
        closest = min(candlesticks, key=lambda x: abs(x['end_period_ts'] - target_ts))
        return closest
    
    def get_9pm_cost(self, series_ticker: str, market_ticker: str, date: datetime) -> Optional[int]:
        """Get the yes_ask.open cost at 9PM the day before"""
        # 9PM the day before
        target_time = datetime.combine(date - timedelta(days=1), time(21, 0))
        target_ts = self.datetime_to_timestamp(target_time)
        
        # Get candlestick data around 9PM
        start_ts = target_ts - 7200  # 2 hours before
        end_ts = target_ts + 7200    # 2 hours after
        
        candlestick_data = self.get_market_candlesticks(
            series_ticker, market_ticker, start_ts, end_ts
        )
        
        if candlestick_data and candlestick_data.get('candlesticks'):
            closest = self.find_exact_candlestick(target_ts, candlestick_data['candlesticks'])
            
            if closest and closest.get('yes_ask') and closest['yes_ask'].get('open') is not None:
                return closest['yes_ask']['open']
        
        return None
    
    def get_9pm_ask_cost(self, series_ticker: str, market_ticker: str, date: datetime) -> Optional[int]:
        """Get the yes_ask.open cost at 9PM the day before"""
        target_time = datetime.combine(date - timedelta(days=1), time(21, 0))
        target_ts = self.datetime_to_timestamp(target_time)
        
        start_ts = target_ts - 7200  # 2 hours before
        end_ts = target_ts + 7200    # 2 hours after
        
        candlestick_data = self.get_market_candlesticks(series_ticker, market_ticker, start_ts, end_ts)
        
        if candlestick_data and candlestick_data.get('candlesticks'):
            closest = self.find_exact_candlestick(target_ts, candlestick_data['candlesticks'])
            if closest and closest.get('yes_ask') and closest['yes_ask'].get('open') is not None:
                return closest['yes_ask']['open']
        return None

    def get_9pm_bid_cost(self, series_ticker: str, market_ticker: str, date: datetime) -> Optional[int]:
        """Get the yes_bid.open cost at 9PM the day before"""
        target_time = datetime.combine(date - timedelta(days=1), time(21, 0))
        target_ts = self.datetime_to_timestamp(target_time)
        
        start_ts = target_ts - 7200  # 2 hours before
        end_ts = target_ts + 7200    # 2 hours after
        
        candlestick_data = self.get_market_candlesticks(series_ticker, market_ticker, start_ts, end_ts)
        
        if candlestick_data and candlestick_data.get('candlesticks'):
            closest = self.find_exact_candlestick(target_ts, candlestick_data['candlesticks'])
            if closest and closest.get('yes_bid') and closest['yes_bid'].get('open') is not None:
                return closest['yes_bid']['open']
        return None

    def get_8am_ask_cost(self, series_ticker: str, market_ticker: str, date: datetime) -> Optional[int]:
        """Get the yes_ask.open cost at 8AM the day of"""
        target_time = datetime.combine(date, time(8, 0))
        target_ts = self.datetime_to_timestamp(target_time)
        
        start_ts = target_ts - 7200  # 2 hours before
        end_ts = target_ts + 7200    # 2 hours after
        
        candlestick_data = self.get_market_candlesticks(series_ticker, market_ticker, start_ts, end_ts)
        
        if candlestick_data and candlestick_data.get('candlesticks'):
            closest = self.find_exact_candlestick(target_ts, candlestick_data['candlesticks'])
            if closest and closest.get('yes_ask') and closest['yes_ask'].get('open') is not None:
                return closest['yes_ask']['open']
        return None

    def get_8am_bid_cost(self, series_ticker: str, market_ticker: str, date: datetime) -> Optional[int]:
        """Get the yes_bid.open cost at 8AM the day of"""
        target_time = datetime.combine(date, time(8, 0))
        target_ts = self.datetime_to_timestamp(target_time)
        
        start_ts = target_ts - 7200  # 2 hours before
        end_ts = target_ts + 7200    # 2 hours after
        
        candlestick_data = self.get_market_candlesticks(series_ticker, market_ticker, start_ts, end_ts)
        
        if candlestick_data and candlestick_data.get('candlesticks'):
            closest = self.find_exact_candlestick(target_ts, candlestick_data['candlesticks'])
            if closest and closest.get('yes_bid') and closest['yes_bid'].get('open') is not None:
                return closest['yes_bid']['open']
        return None
        """Get the yes_ask.open cost at 8AM the day of"""
        # 8AM the day of the event
        target_time = datetime.combine(date, time(8, 0))
        target_ts = self.datetime_to_timestamp(target_time)
        
        print(f"        Getting 8AM cost for {market_ticker} at {target_time} (ts: {target_ts})")
        
        # Get candlestick data around 8AM
        start_ts = target_ts - 7200  # 2 hours before
        end_ts = target_ts + 7200    # 2 hours after
        
        candlestick_data = self.get_market_candlesticks(
            series_ticker, market_ticker, start_ts, end_ts
        )
        
        if candlestick_data and candlestick_data.get('candlesticks'):
            candlesticks = candlestick_data['candlesticks']
            print(f"        Found {len(candlesticks)} candlesticks")
            
            closest = self.find_exact_candlestick(target_ts, candlesticks)
            
            if closest:
                timestamp = closest.get('end_period_ts')
                yes_ask_data = closest.get('yes_ask', {})
                ask_open = yes_ask_data.get('open')
                ask_close = yes_ask_data.get('close')
                
                print(f"        Closest candlestick: ts={timestamp}")
                print(f"        yes_ask.open: {ask_open}, yes_ask.close: {ask_close}")
                
                if ask_open is not None:
                    return ask_open
                else:
                    print(f"        No yes_ask.open data")
            else:
                print(f"        No candlestick found for target time")
        else:
            print(f"        No candlestick data returned")
        
        return None
    
    def get_ending_cost(self, series_ticker: str, market_ticker: str) -> Optional[int]:
        """Get the final yes_ask.open cost from the last candlestick"""
        try:
            # Get a wide time range to capture the final data
            end_time = datetime.now()
            start_time = end_time - timedelta(days=30)
            
            candlestick_data = self.get_market_candlesticks(
                series_ticker, 
                market_ticker,
                self.datetime_to_timestamp(start_time),
                self.datetime_to_timestamp(end_time)
            )
            
            if candlestick_data and candlestick_data.get('candlesticks'):
                # Get the last candlestick
                last_candle = candlestick_data['candlesticks'][-1]
                if last_candle and last_candle.get('yes_ask') and last_candle['yes_ask'].get('open') is not None:
                    return last_candle['yes_ask']['open']
            
            return None
        except Exception as e:
            print(f"Error getting ending cost for {market_ticker}: {str(e)}")
            return None
    
    def analyze_9pm_highest_ask(self, city: str, date: datetime) -> Optional[Dict]:
        """Find the market with HIGHEST ask price at 9PM and track its performance"""
        event_ticker = self.create_event_ticker(city, date)
        series_ticker = self.cities[city]
        
        event_data = self.get_event_markets(event_ticker)
        if not event_data or not event_data.get('markets'):
            return None
        
        best_market = None
        highest_ask = 0
        
        print(f"    Comparing ASK prices for {city}:")
        
        for market in event_data['markets']:
            market_ticker = market['ticker']
            temp_range = market.get('subtitle', 'Unknown')
            result = market.get('result', 'unknown')
            
            ask_cost = self.get_9pm_ask_cost(series_ticker, market_ticker, date)
            
            if ask_cost is not None:
                print(f"      {temp_range}: ASK=${ask_cost}")
                
                if ask_cost > highest_ask:
                    highest_ask = ask_cost
                    ending_cost = self.get_ending_cost(series_ticker, market_ticker)
                    won = (result == 'yes')
                    
                    print(f"      ↑ NEW HIGHEST ASK: ${ask_cost} ({temp_range}) - RESULT: '{result}' → {('WON' if won else 'LOST')}")
                    
                    best_market = {
                        'city': city,
                        'date': date.strftime('%b-%d-%Y').upper(),
                        'temp_range': temp_range,
                        'cost_9pm': ask_cost,
                        'ending_cost': ending_cost if ending_cost is not None else 0,
                        'won': 'Won' if won else 'Lost',
                        'ticker': market_ticker
                    }
            else:
                print(f"      {temp_range}: No ASK data")
        
        if best_market:
            print(f"    ✓ FINAL WINNER: {best_market['temp_range']} ASK=${best_market['cost_9pm']} → {best_market['won']}")
        
        return best_market

    def analyze_9pm_highest_bid(self, city: str, date: datetime) -> Optional[Dict]:
        """Find the market with HIGHEST bid price at 9PM and track its performance"""
        event_ticker = self.create_event_ticker(city, date)
        series_ticker = self.cities[city]
        
        event_data = self.get_event_markets(event_ticker)
        if not event_data or not event_data.get('markets'):
            return None
        
        best_market = None
        highest_bid = 0
        
        print(f"    Comparing BID prices for {city}:")
        
        for market in event_data['markets']:
            market_ticker = market['ticker']
            temp_range = market.get('subtitle', 'Unknown')
            result = market.get('result', 'unknown')
            
            bid_cost = self.get_9pm_bid_cost(series_ticker, market_ticker, date)
            
            if bid_cost is not None:
                print(f"      {temp_range}: BID=${bid_cost}")
                
                if bid_cost > highest_bid:
                    highest_bid = bid_cost
                    ending_cost = self.get_ending_cost(series_ticker, market_ticker)
                    won = (result == 'yes')
                    
                    print(f"      ↑ NEW HIGHEST BID: ${bid_cost} ({temp_range}) - RESULT: '{result}' → {('WON' if won else 'LOST')}")
                    
                    best_market = {
                        'city': city,
                        'date': date.strftime('%b-%d-%Y').upper(),
                        'temp_range': temp_range,
                        'cost_9pm': bid_cost,
                        'ending_cost': ending_cost if ending_cost is not None else 0,
                        'won': 'Won' if won else 'Lost',
                        'ticker': market_ticker
                    }
            else:
                print(f"      {temp_range}: No BID data")
        
        if best_market:
            print(f"    ✓ FINAL WINNER: {best_market['temp_range']} BID=${best_market['cost_9pm']} → {best_market['won']}")
        
        return best_market

    def analyze_8am_highest_ask(self, city: str, date: datetime) -> Optional[Dict]:
        """Find the market with HIGHEST ask price at 8AM and track its performance"""
        event_ticker = self.create_event_ticker(city, date)
        series_ticker = self.cities[city]
        
        event_data = self.get_event_markets(event_ticker)
        if not event_data or not event_data.get('markets'):
            return None
        
        best_market = None
        highest_ask = 0
        
        print(f"    Comparing ASK prices for {city}:")
        
        for market in event_data['markets']:
            market_ticker = market['ticker']
            temp_range = market.get('subtitle', 'Unknown')
            result = market.get('result', 'unknown')
            
            ask_cost = self.get_8am_ask_cost(series_ticker, market_ticker, date)
            
            if ask_cost is not None:
                print(f"      {temp_range}: ASK=${ask_cost}")
                
                if ask_cost > highest_ask:
                    highest_ask = ask_cost
                    ending_cost = self.get_ending_cost(series_ticker, market_ticker)
                    won = (result == 'yes')
                    
                    print(f"      ↑ NEW HIGHEST ASK: ${ask_cost} ({temp_range}) - RESULT: '{result}' → {('WON' if won else 'LOST')}")
                    
                    best_market = {
                        'city': city,
                        'date': date.strftime('%b-%d-%Y').upper(),
                        'temp_range': temp_range,
                        'cost_8am': ask_cost,
                        'ending_cost': ending_cost if ending_cost is not None else 0,
                        'won': 'Won' if won else 'Lost',
                        'ticker': market_ticker
                    }
            else:
                print(f"      {temp_range}: No ASK data")
        
        if best_market:
            print(f"    ✓ FINAL WINNER: {best_market['temp_range']} ASK=${best_market['cost_8am']} → {best_market['won']}")
        
        return best_market

    def analyze_8am_highest_bid(self, city: str, date: datetime) -> Optional[Dict]:
        """Find the market with HIGHEST bid price at 8AM and track its performance"""
        event_ticker = self.create_event_ticker(city, date)
        series_ticker = self.cities[city]
        
        event_data = self.get_event_markets(event_ticker)
        if not event_data or not event_data.get('markets'):
            return None
        
        best_market = None
        highest_bid = 0
        
        print(f"    Comparing BID prices for {city}:")
        
        for market in event_data['markets']:
            market_ticker = market['ticker']
            temp_range = market.get('subtitle', 'Unknown')
            result = market.get('result', 'unknown')
            
            bid_cost = self.get_8am_bid_cost(series_ticker, market_ticker, date)
            
            if bid_cost is not None:
                print(f"      {temp_range}: BID=${bid_cost}")
                
                if bid_cost > highest_bid:
                    highest_bid = bid_cost
                    ending_cost = self.get_ending_cost(series_ticker, market_ticker)
                    won = (result == 'yes')
                    
                    print(f"      ↑ NEW HIGHEST BID: ${bid_cost} ({temp_range}) - RESULT: '{result}' → {('WON' if won else 'LOST')}")
                    
                    best_market = {
                        'city': city,
                        'date': date.strftime('%b-%d-%Y').upper(),
                        'temp_range': temp_range,
                        'cost_8am': bid_cost,
                        'ending_cost': ending_cost if ending_cost is not None else 0,
                        'won': 'Won' if won else 'Lost',
                        'ticker': market_ticker
                    }
            else:
                print(f"      {temp_range}: No BID data")
        
        if best_market:
            print(f"    ✓ FINAL WINNER: {best_market['temp_range']} BID=${best_market['cost_8am']} → {best_market['won']}")
        
        return best_market
        """Find the most expensive (highest ask) market at 9PM and track its performance"""
        event_ticker = self.create_event_ticker(city, date)
        series_ticker = self.cities[city]
        
        event_data = self.get_event_markets(event_ticker)
        if not event_data:
            return None
        
        markets = event_data.get('markets', [])
        if not markets:
            return None
        
        best_market = None
        highest_9pm_cost = 0  # Find HIGHEST ask price
        
        # Find the market with HIGHEST 9PM cost (most expensive/least popular)
        for market in markets:
            market_ticker = market['ticker']
            temp_range = market.get('subtitle', 'Unknown')
            result = market.get('result', 'unknown')
            
            cost_9pm = self.get_9pm_cost(series_ticker, market_ticker, date)
            
            if cost_9pm is not None and cost_9pm > highest_9pm_cost:
                highest_9pm_cost = cost_9pm
                
                # Get ending cost
                ending_cost = self.get_ending_cost(series_ticker, market_ticker)
                
                # Determine if it won
                won = (result == 'yes')
                
                print(f"      Market result field: '{result}' -> Won: {won}")
                
                best_market = {
                    'city': city,
                    'date': date.strftime('%b-%d-%Y').upper(),
                    'temp_range': temp_range,
                    'cost_9pm': cost_9pm,
                    'ending_cost': ending_cost if ending_cost is not None else 0,
                    'won': 'Won' if won else 'Lost',
                    'ticker': market_ticker,
                    'result_field': result  # For debugging
                }
        
        return best_market
    
    def analyze_8am_each_city(self, date: datetime) -> List[Dict]:
        """Find the most expensive market for EACH city at 8AM"""
        print(f"  8AM Analysis for {date.strftime('%Y-%m-%d')} - finding HIGHEST ask price per city")
        
        city_results = []
        
        # Check each city individually
        for city, series_ticker in self.cities.items():
            print(f"    Analyzing {city}...")
            event_ticker = self.create_event_ticker(city, date)
            event_data = self.get_event_markets(event_ticker)
            
            if not event_data or not event_data.get('markets'):
                print(f"      No event data for {city}")
                continue
            
            print(f"      Found {len(event_data['markets'])} markets for {city}")
            
            # Find HIGHEST 8AM cost for THIS city
            max_8am_cost = 0
            best_market = None
            
            for market in event_data['markets']:
                market_ticker = market['ticker']
                temp_range = market.get('subtitle', 'Unknown')
                result = market.get('result', 'unknown')
                
                cost_8am = self.get_8am_cost(series_ticker, market_ticker, date)
                
                if cost_8am is not None:
                    print(f"        {temp_range}: 8AM cost = ${cost_8am}")
                    
                    if cost_8am > max_8am_cost:
                        max_8am_cost = cost_8am
                        ending_cost = self.get_ending_cost(series_ticker, market_ticker)
                        won = (result == 'yes')
                        
                        print(f"        NEW HIGHEST for {city}: ${cost_8am} ({temp_range}) - Result: {result}")
                        
                        best_market = {
                            'city': city,
                            'date': date.strftime('%b-%d-%Y').upper(),
                            'temp_range': temp_range,
                            'cost_8am': cost_8am,
                            'ending_cost': ending_cost if ending_cost is not None else 0,
                            'won': 'Won' if won else 'Lost',
                            'ticker': market_ticker,
                            'result_field': result
                        }
            
            if best_market:
                city_results.append(best_market)
                print(f"      {city} FINAL: ${best_market['cost_8am']} ({best_market['temp_range']}) - {best_market['won']}")
            else:
                print(f"      No 8AM data found for {city}")
        
        return city_results
        """Find the most expensive city at 8AM and track its performance"""
        print(f"  8AM Analysis for {date.strftime('%Y-%m-%d')} - looking for HIGHEST ask prices")
        
        city_costs = {}
        
        # Check each city to find most expensive (highest cost)
        for city, series_ticker in self.cities.items():
            print(f"    Checking city: {city}")
            event_ticker = self.create_event_ticker(city, date)
            event_data = self.get_event_markets(event_ticker)
            
            if not event_data or not event_data.get('markets'):
                print(f"      No event data for {city}")
                continue
            
            print(f"      Found {len(event_data['markets'])} markets for {city}")
            
            # Find HIGHEST 8AM cost across all markets for this city
            max_8am_cost = 0  # Find HIGHEST ask price
            best_city_market = None
            markets_checked = 0
            
            for market in event_data['markets']:
                market_ticker = market['ticker']
                temp_range = market.get('subtitle', 'Unknown')
                result = market.get('result', 'unknown')
                
                cost_8am = self.get_8am_cost(series_ticker, market_ticker, date)
                markets_checked += 1
                
                if cost_8am is not None:
                    print(f"      {temp_range}: 8AM cost = ${cost_8am}")
                    
                    if cost_8am > max_8am_cost:
                        max_8am_cost = cost_8am
                        ending_cost = self.get_ending_cost(series_ticker, market_ticker)
                        won = (result == 'yes')
                        
                        print(f"      NEW HIGHEST for {city}: ${cost_8am} ({temp_range}) - Result: {result}")
                        
                        best_city_market = {
                            'city': city,
                            'cost_8am': cost_8am,
                            'ending_cost': ending_cost if ending_cost is not None else 0,
                            'won': won,
                            'temp_range': temp_range,
                            'ticker': market_ticker,
                            'result_field': result
                        }
                else:
                    print(f"      {temp_range}: No 8AM cost data")
            
            print(f"      {city} summary: checked {markets_checked} markets, highest cost: ${max_8am_cost}")
            print(f"      Best market stored: {best_city_market['temp_range'] if best_city_market else 'None'} at ${best_city_market['cost_8am'] if best_city_market else 'None'}")
            
            if best_city_market:
                city_costs[city] = best_city_market
        
        print(f"  Cities with 8AM data: {list(city_costs.keys())}")
        
        if city_costs:
            # Find city with HIGHEST 8AM cost (most expensive/least popular)
            print(f"  Comparing cities:")
            for city, data in city_costs.items():
                print(f"    {city}: ${data['cost_8am']} ({data['temp_range']}) - {data['result_field']}")
            
            most_expensive_city = max(city_costs.items(), key=lambda x: x[1]['cost_8am'])
            best_data = most_expensive_city[1]
            
            print(f"  FINAL WINNER: {best_data['city']} with ${best_data['cost_8am']} ({best_data['temp_range']}) - {best_data['result_field']}")
            
            return {
                'city': best_data['city'],
                'date': date.strftime('%b-%d-%Y').upper(),
                'temp_range': best_data['temp_range'],
                'cost_8am': best_data['cost_8am'],
                'ending_cost': best_data['ending_cost'],
                'won': 'Won' if best_data['won'] else 'Lost',
                'ticker': best_data['ticker']
            }
        
        print(f"  No 8AM data found for any city")
        return None
    
    def process_date_range(self, start_date: datetime, end_date: datetime):
        """Process all dates in the given range"""
        current_date = start_date
        total_days = (end_date - start_date).days + 1
        processed_days = 0
        
        print(f"Processing {total_days} days from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
        
        while current_date <= end_date:
            processed_days += 1
            print(f"Processing {current_date.strftime('%Y-%m-%d')} ({processed_days}/{total_days})")
            
            # 9PM Ask Analysis - Highest ask price for each city
            for city in self.cities.keys():
                print(f"  Analyzing 9PM ASK for {city}...")
                result_9pm_ask = self.analyze_9pm_highest_ask(city, current_date)
                if result_9pm_ask:
                    self.pm9_ask_results.append(result_9pm_ask)
                    print(f"    ASK: {result_9pm_ask['temp_range']}: 9PM=${result_9pm_ask['cost_9pm']}, End=${result_9pm_ask['ending_cost']}, {result_9pm_ask['won']}")
                else:
                    print(f"    No ASK data found for {city}")
            
            # 9PM Bid Analysis - Highest bid price for each city
            for city in self.cities.keys():
                print(f"  Analyzing 9PM BID for {city}...")
                result_9pm_bid = self.analyze_9pm_highest_bid(city, current_date)
                if result_9pm_bid:
                    self.pm9_bid_results.append(result_9pm_bid)
                    print(f"    BID: {result_9pm_bid['temp_range']}: 9PM=${result_9pm_bid['cost_9pm']}, End=${result_9pm_bid['ending_cost']}, {result_9pm_bid['won']}")
                else:
                    print(f"    No BID data found for {city}")
            
            # 8AM Ask Analysis - Highest ask price for each city
            for city in self.cities.keys():
                print(f"  Analyzing 8AM ASK for {city}...")
                result_8am_ask = self.analyze_8am_highest_ask(city, current_date)
                if result_8am_ask:
                    self.am8_ask_results.append(result_8am_ask)
                    print(f"    ASK: {result_8am_ask['temp_range']}: 8AM=${result_8am_ask['cost_8am']}, End=${result_8am_ask['ending_cost']}, {result_8am_ask['won']}")
                else:
                    print(f"    No 8AM ASK data found for {city}")
            
            # 8AM Bid Analysis - Highest bid price for each city
            for city in self.cities.keys():
                print(f"  Analyzing 8AM BID for {city}...")
                result_8am_bid = self.analyze_8am_highest_bid(city, current_date)
                if result_8am_bid:
                    self.am8_bid_results.append(result_8am_bid)
                    print(f"    BID: {result_8am_bid['temp_range']}: 8AM=${result_8am_bid['cost_8am']}, End=${result_8am_bid['ending_cost']}, {result_8am_bid['won']}")
                else:
                    print(f"    No 8AM BID data found for {city}")
            
            # Rate limiting
            time_module.sleep(0.5)
            current_date += timedelta(days=1)
    
    def save_results(self, force_save=False):
        """Save results to 4 separate CSV files"""
        if not force_save:
            # Only save if we have meaningful data to avoid empty files
            min_records = 5
            if (len(self.pm9_ask_results) < min_records and 
                len(self.pm9_bid_results) < min_records and 
                len(self.am8_ask_results) < min_records and 
                len(self.am8_bid_results) < min_records):
                print("Not enough data to save yet...")
                return
        
        # 9PM Ask Results
        if self.pm9_ask_results:
            df_9pm_ask = pd.DataFrame([{
                'City': r['city'],
                'Date': r['date'],
                'Temp_Range': r['temp_range'],
                '9PM_Ask': r['cost_9pm'],
                'Ending_Cost': r['ending_cost'],
                'Won': r['won']
            } for r in self.pm9_ask_results])
            
            df_9pm_ask.to_csv('kalshi_weather_9pm_ask_2024.csv', index=False)
            print(f"✓ Saved 9PM ASK: {len(self.pm9_ask_results)} records")
        
        # 9PM Bid Results
        if self.pm9_bid_results:
            df_9pm_bid = pd.DataFrame([{
                'City': r['city'],
                'Date': r['date'],
                'Temp_Range': r['temp_range'],
                '9PM_Bid': r['cost_9pm'],
                'Ending_Cost': r['ending_cost'],
                'Won': r['won']
            } for r in self.pm9_bid_results])
            
            df_9pm_bid.to_csv('kalshi_weather_9pm_bid_2024.csv', index=False)
            print(f"✓ Saved 9PM BID: {len(self.pm9_bid_results)} records")
        
        # 8AM Ask Results
        if self.am8_ask_results:
            df_8am_ask = pd.DataFrame([{
                'City': r['city'],
                'Date': r['date'],
                'Temp_Range': r['temp_range'],
                '8AM_Ask': r['cost_8am'],
                'Ending_Cost': r['ending_cost'],
                'Won': r['won']
            } for r in self.am8_ask_results])
            
            df_8am_ask.to_csv('kalshi_weather_8am_ask_2024.csv', index=False)
            print(f"✓ Saved 8AM ASK: {len(self.am8_ask_results)} records")
        
        # 8AM Bid Results
        if self.am8_bid_results:
            df_8am_bid = pd.DataFrame([{
                'City': r['city'],
                'Date': r['date'],
                'Temp_Range': r['temp_range'],
                '8AM_Bid': r['cost_8am'],
                'Ending_Cost': r['ending_cost'],
                'Won': r['won']
            } for r in self.am8_bid_results])
            
            df_8am_bid.to_csv('kalshi_weather_8am_bid_2024.csv', index=False)
            print(f"✓ Saved 8AM BID: {len(self.am8_bid_results)} records")

    def save_progress_checkpoint(self, current_date: datetime):
        """Save current progress to a checkpoint file"""
        checkpoint = {
            'last_processed_date': current_date.strftime('%Y-%m-%d'),
            'pm9_ask_count': len(self.pm9_ask_results),
            'pm9_bid_count': len(self.pm9_bid_results),
            'am8_ask_count': len(self.am8_ask_results),
            'am8_bid_count': len(self.am8_bid_results),
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        with open('kalshi_progress_checkpoint.json', 'w') as f:
            json.dump(checkpoint, f, indent=2)
        
        print(f"📍 Checkpoint saved: {current_date.strftime('%Y-%m-%d')}")

    def load_progress_checkpoint(self) -> Optional[datetime]:
        """Load the last checkpoint and return the date to resume from"""
        try:
            with open('kalshi_progress_checkpoint.json', 'r') as f:
                checkpoint = json.load(f)
            
            last_date = datetime.strptime(checkpoint['last_processed_date'], '%Y-%m-%d')
            print(f"📍 Found checkpoint: {checkpoint['last_processed_date']}")
            print(f"   Records so far: 9PM_ASK={checkpoint['pm9_ask_count']}, 9PM_BID={checkpoint['pm9_bid_count']}")
            print(f"                   8AM_ASK={checkpoint['am8_ask_count']}, 8AM_BID={checkpoint['am8_bid_count']}")
            
            # Resume from the next day
            resume_date = last_date + timedelta(days=1)
            return resume_date
            
        except FileNotFoundError:
            print("📍 No checkpoint found, starting fresh")
            return None
        except Exception as e:
            print(f"📍 Error loading checkpoint: {e}")
            return None
    
    def test_single_event(self, city: str = 'NY', test_date: str = '2024-07-20'):
        """Test function to verify API calls work with a known event"""
        date_obj = datetime.strptime(test_date, '%Y-%m-%d')
        event_ticker = self.create_event_ticker(city, date_obj)
        
        print(f"Testing with event: {event_ticker}")
        
        event_data = self.get_event_markets(event_ticker)
        if event_data:
            print(f"✓ Found {len(event_data.get('markets', []))} markets")
            return True
        else:
            print("✗ No event data found")
            return False
    
    def run_full_analysis_2024(self):
        """Run the complete analysis for 2024"""
        start_date = datetime(2024, 1, 1)
        end_date = datetime(2024, 10, 31)
        
        print("Starting Kalshi Weather Market Analysis for 2024...")
        self.process_date_range(start_date, end_date)
        self.save_results()
        
        print("\nAnalysis Summary:")
        print(f"9PM ASK Analysis: {len(self.pm9_ask_results)} records")
        print(f"9PM BID Analysis: {len(self.pm9_bid_results)} records")
        print(f"8AM ASK Analysis: {len(self.am8_ask_results)} records")
        print(f"8AM BID Analysis: {len(self.am8_bid_results)} records")

# Initialize and run the analyzer
analyzer = KalshiWeatherAnalyzer()

# Test API connectivity first
print("Testing API connectivity with known 2024 event...")
if analyzer.test_single_event('NY', '2024-07-20'):
    print("✓ API test successful!")
    
    print("\n" + "="*60)
    print("RUNNING SIMPLE EFFICIENCY ANALYSIS")
    print("="*60)
    
    # Test with just July 20-22, 2024
    test_start = datetime(2024, 7, 20)
    test_end = datetime(2024, 7, 22)
    analyzer.process_date_range(test_start, test_end)
    analyzer.save_results()
    
    print("\n" + "="*60)
    print("Sample CSV Output Format:")
    print("="*60)
    print("City, Date, Temp_Range, 9PM_Cost, Ending_Cost, Won")
    print("MIA, JUL-20-2024, 95° or above, 85, 0, Lost")
    print("NY, JUL-20-2024, 88° to 89°, 92, 1, Lost")
    print("CHI, JUL-21-2024, 85° to 86°, 78, 99, Won")
    
else:
    print("✗ API test failed - check connectivity and endpoints")

# To run full analysis, uncomment:
# analyzer.run_full_analysis_2024()

Testing API connectivity with known 2024 event...
Testing with event: HIGHNY-24JUL20
✓ Found 6 markets
✓ API test successful!

RUNNING SIMPLE EFFICIENCY ANALYSIS
Processing 3 days from 2024-07-20 to 2024-07-22
Processing 2024-07-20 (1/3)
  Analyzing 9PM ASK for NY...
    Comparing ASK prices for NY:
      81° or below: ASK=$12
      ↑ NEW HIGHEST ASK: $12 (81° or below) - RESULT: 'no' → LOST
      82° to 83°: ASK=$15
      ↑ NEW HIGHEST ASK: $15 (82° to 83°) - RESULT: 'yes' → WON
      84° to 85°: ASK=$39
      ↑ NEW HIGHEST ASK: $39 (84° to 85°) - RESULT: 'no' → LOST
      86° to 87°: ASK=$40
      ↑ NEW HIGHEST ASK: $40 (86° to 87°) - RESULT: 'no' → LOST
      88° to 89°: ASK=$15
      90° or above: No ASK data
    ✓ FINAL WINNER: 86° to 87° ASK=$40 → Lost
    ASK: 86° to 87°: 9PM=$40, End=$0, Lost
  Analyzing 9PM ASK for LA...
Failed to get event HIGHLA-24JUL20: 404
    No ASK data found for LA
  Analyzing 9PM ASK for PHIL...
Failed to get event HIGHPHIL-24JUL20: 404
    No ASK data

In [4]:
analyzer.run_full_analysis_2024()

Starting Kalshi Weather Market Analysis for 2024...
Processing 305 days from 2024-01-01 to 2024-10-31
Processing 2024-01-01 (1/305)
  Analyzing 9PM ASK for NY...
    Comparing ASK prices for NY:
      41° or below: ASK=$93
      ↑ NEW HIGHEST ASK: $93 (41° or below) - RESULT: 'no' → LOST
      42° to 43°: ASK=$95
      ↑ NEW HIGHEST ASK: $95 (42° to 43°) - RESULT: 'no' → LOST
      44° to 45°: ASK=$95
      46° to 47°: ASK=$93
      48° to 49°: No ASK data
      50° or above: No ASK data
    ✓ FINAL WINNER: 42° to 43° ASK=$95 → Lost
    ASK: 42° to 43°: 9PM=$95, End=$0, Lost
  Analyzing 9PM ASK for LA...
Failed to get event HIGHLA-24JAN01: 404
    No ASK data found for LA
  Analyzing 9PM ASK for PHIL...
Failed to get event HIGHPHIL-24JAN01: 404
    No ASK data found for PHIL
  Analyzing 9PM ASK for CHI...
    Comparing ASK prices for CHI:
      30° or below: ASK=$84
      ↑ NEW HIGHEST ASK: $84 (30° or below) - RESULT: 'no' → LOST
      31° to 32°: ASK=$91
      ↑ NEW HIGHEST ASK: $91 